## Hsf1 binding motif

We have a table of Hsf1 ChIP-Seq peaks, including the name of the chromosome and the position of the "summit" within the peak.

In earlier classes, we learned how to read in the yeast genome sequence.

Today, we'll look up the genome sequence around each ChIP-Seq summit and write our own Fasta-format file that we can use as input to MEME, the motif search tool.

We'll use pandas quite a bit here.

In [2]:
import pandas as pd

We saw last time that the "peaks" file from MACS contained all the information we wanted, with nicely named columns.

We'll read in the peaks file `ChIP_1M_peaks.xls` as a pandas `DataFrame` and store it in the `peaks` variable

In [3]:
peaks = pd.read_csv("macs2/ChIP_1M_peaks.xls", comment='#', delimiter='\t')

Use `sort_values()` to sort peaks according to their significance. Start with the most significant peak first, using the `ascending` parameter.

In [4]:
peaks = peaks.sort_values("-log10(qvalue)", ascending=False)

We can use the `head` function to take only the "best" peaks. We'll test our code on the best 5 peaks from the mini ChIP-Seq data set here in class today.

In [5]:
peaks.head(5)

,chr,start,end,length,abs_summit,pileup,-log10(pvalue),fold_enrichment,-log10(qvalue),name
1,chrI,141519,141990,472,141773,105,88.3149,14.6006,81.2301,ChIP_1M_peak_2
82,chrXII,489785,490557,773,490213,86,69.5089,13.1420,64.4996,ChIP_1M_peak_83
47,chrVII,914828,915214,387,915020,59,54.4093,14.9254,49.9879,ChIP_1M_peak_48
36,chrVI,210186,210570,385,210372,53,50.5849,15.2542,46.2428,ChIP_1M_peak_37
44,chrVII,771903,772374,472,772134,48,38.9013,11.4486,34.7470,ChIP_1M_peak_45


We will extract the DNA sequence in a somewhat arbitrary window of ~100 base pairs, 50 on each side of the actual summit.

To do this, define two new columns in the `peaks` data frame, one containing the start of the motif search region and the other containing the end of the region. We'll call these `motif_start` and `motif_end`

In [6]:
peaks["motif_start"] = peaks["abs_summit"] - 50
peaks["motif_end"] = peaks["abs_summit"] + 50
peaks.head()

,chr,start,end,length,abs_summit,pileup,-log10(pvalue),fold_enrichment,-log10(qvalue),name,motif_start,motif_end
1,chrI,141519,141990,472,141773,105,88.3149,14.6006,81.2301,ChIP_1M_peak_2,141723,141823
82,chrXII,489785,490557,773,490213,86,69.5089,13.1420,64.4996,ChIP_1M_peak_83,490163,490263
47,chrVII,914828,915214,387,915020,59,54.4093,14.9254,49.9879,ChIP_1M_peak_48,914970,915070
36,chrVI,210186,210570,385,210372,53,50.5849,15.2542,46.2428,ChIP_1M_peak_37,210322,210422
44,chrVII,771903,772374,472,772134,48,38.9013,11.4486,34.7470,ChIP_1M_peak_45,772084,772184


We want to _loop_ over each row of the `DataFrame`.

The `itertuples()` method gives us an iterator over the rows. Each row is a `namedtuple` with named fields taken from the column names in the DataFrame.

To illustrate this, loop over each peak and print the name and the location (chromosome and start to end coordinates).

In [8]:
for peak in peaks.head().itertuples():
    print('Peak {} on chrom {} starting at {} and ending at {}'
          .format(peak.name, peak.chr, peak.motif_start, peak.motif_end))

Peak ChIP_1M_peak_2 on chrom chrI starting at 141723 and ending at 141823
Peak ChIP_1M_peak_83 on chrom chrXII starting at 490163 and ending at 490263
Peak ChIP_1M_peak_48 on chrom chrVII starting at 914970 and ending at 915070
Peak ChIP_1M_peak_37 on chrom chrVI starting at 210322 and ending at 210422
Peak ChIP_1M_peak_45 on chrom chrVII starting at 772084 and ending at 772184


To extract the sequences from these regions of the genome, we need to refer back to the yeast genome sequence.

Below, we import `SeqIO` from the `Bio` package.

I've also provided the name of the fasta file of chromosome sequences.
```
chrom_file = '/home/jovyan/shared/MCB280A_data/S288C_R64-3-1/S288C_reference_sequence_R64-3-1_20210421.fsa'
```

In [9]:
from Bio import SeqIO

chrom_file = '/home/jovyan/shared/MCB280A_data/S288C_R64-3-1/S288C_reference_sequence_R64-3-1_20210421.fsa'

As a starting point, we'll extract the sequence for one _manually defined_ genomic region.
```
motif_chr = 'chrI'
motif_start = 141723
motif_end = 141823
```

We'll loop through each chromosome, test whether its name (the `.name` field) matches our chromosome of interest, and then slice out the desired region from the sequence (in the `.seq` field).

In [10]:
motif_chr = 'chrI'
motif_start = 141723
motif_end = 141823

chroms = SeqIO.parse(chrom_file, 'fasta')
for chrom in chroms:
    if chrom.name == motif_chr:
        site = chrom.seq[motif_start:motif_end]
        print(str(site))

AAAATCTGGAGAAAAAACGCAAATGACAGCTTCTAAACGTTCCGTGTGCTTTCTTTCTAGAATGTTCTGGAAAGTTTACAACAATCCACAAGAACGAAAA


Next, instead of manually defining the region, we'll loop over the rows in `peaks` and use the values in the table for each sequence.

In [11]:
for peak in peaks.head().itertuples():
    chroms = SeqIO.parse(chrom_file, 'fasta')
    for chrom in chroms:
        if chrom.name == peak.chr:
            site = chrom.seq[peak.motif_start:peak.motif_end]
            print('{}\t{}'.format(peak.name, site))

ChIP_1M_peak_2	AAAATCTGGAGAAAAAACGCAAATGACAGCTTCTAAACGTTCCGTGTGCTTTCTTTCTAGAATGTTCTGGAAAGTTTACAACAATCCACAAGAACGAAAA
ChIP_1M_peak_83	AGGGTAAAACTAACCTGTCTCACGACGGTCTAAACCCAGCTCACGTTCCCTATTAGTGGGTGAACAATCCAACGCTTACCGAATTCTGCTTCGGTATGAT
ChIP_1M_peak_48	AAAAGCCCTTACTAACCCTACAATAAATTGTGCCGAAACCCTCTGGAGTTTTCTAGAATATTCTAGCCCCATCAGGGCTAGAATATCCTAAAAGTTTATA
ChIP_1M_peak_37	TCACGTTATCAGGCTCATAGCTTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGATTGTTGTTCTAGTCG
ChIP_1M_peak_45	GTTGTAAAGTTACTGACACTTTTTTTTCTAGAAAGTTCCGGAAAATTGCGACACTCGGTGGAGCTCGAGAGTTGTATCCAGTTTTCTTGTTCGGCGATAT


This works okay, but it's inefficient because we read the genome file once for each peak. Instead, we can read the genome just once and store the sequence in a `dict`. We'll use the chromosome names as the _key_ and the sequence as the _value_.

Build this dictionary in a variable named `genome`.

In [14]:
genome = {}
chroms = SeqIO.parse(chrom_file, 'fasta')
for chrom in chroms:
    genome[chrom.name] = chrom.seq

Now, we can use the `.get()` method on the `genome` dictionary to look up any chromosome sequence we want, very quickly.

Adapt the loop above for this more efficient 

In [16]:
for peak in peaks.head().itertuples():
    chrom_seq = genome.get(peak.chr)
    site = chrom_seq[peak.motif_start:peak.motif_end]
    print('{}\t{}'.format(peak.name, site))

ChIP_1M_peak_2	AAAATCTGGAGAAAAAACGCAAATGACAGCTTCTAAACGTTCCGTGTGCTTTCTTTCTAGAATGTTCTGGAAAGTTTACAACAATCCACAAGAACGAAAA
ChIP_1M_peak_83	AGGGTAAAACTAACCTGTCTCACGACGGTCTAAACCCAGCTCACGTTCCCTATTAGTGGGTGAACAATCCAACGCTTACCGAATTCTGCTTCGGTATGAT
ChIP_1M_peak_48	AAAAGCCCTTACTAACCCTACAATAAATTGTGCCGAAACCCTCTGGAGTTTTCTAGAATATTCTAGCCCCATCAGGGCTAGAATATCCTAAAAGTTTATA
ChIP_1M_peak_37	TCACGTTATCAGGCTCATAGCTTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGATTGTTGTTCTAGTCG
ChIP_1M_peak_45	GTTGTAAAGTTACTGACACTTTTTTTTCTAGAAAGTTCCGGAAAATTGCGACACTCGGTGGAGCTCGAGAGTTGTATCCAGTTTTCTTGTTCGGCGATAT


Now, let's write these sequences into our own Fasta file.

We'll need to create our own `Seq` and `SeqRecord` objects. To do this, we'll need to import some more parts of biopython:
```
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
```

In [17]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

After doing this, we can create our own sequence object by writing `Seq(...)`

In [18]:
Seq("ACGTACGT")

Seq('ACGTACGT')

We can create our own sequence record—a sequence object, plus a _name_ in the `id` field, as well.

In [23]:
SeqRecord(Seq("ACGTACGT"), id="MySeq", description='')

SeqRecord(seq=Seq('ACGTACGT'), id='MySeq', name='<unknown name>', description='', dbxrefs=[])

Finally, we can use the `SeqIO.write()` function to write a list of `SeqRecord`s into a file. This function has three parameters:
```
SeqIO.write(list-of-sequences, filename, format)
```

In [24]:
rec1 = SeqRecord(Seq("ACGTACGT"), id="MySeq", description='')
rec2 = SeqRecord(Seq("GCATGCAT"), id="AnotherSeq", description='')
SeqIO.write([rec1, rec2], "test.fa", "fasta")

2

Now, we will construct a list called `sites` containing a `SeqRecord` for each ChIP-Seq peak. Then, we'll write the list into a file called `"Hsf1_sites.fa"`. 

In [26]:
sites = []

for peak in peaks.head().itertuples():
    chrom_seq = genome.get(peak.chr)
    site = chrom_seq[peak.motif_start:peak.motif_end]
    rec = SeqRecord(site, id=peak.name, description='{}:{}-{}'.format(peak.chr, peak.motif_start, peak.motif_end))
    sites.append(rec)
    
SeqIO.write(sites, "Hsf1_sites.fa", "fasta")

5